In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
import numpy as np
from spacy.en import English
import string
import unicodedata

from collections import Counter
from itertools import chain

parser = English()



sqd_data = pd.read_json('/home/anjana/Downloads/train-v1.1.json')

In [2]:
import gensim
import re

import spacy
nlp = spacy.load('en') 

In [3]:
sqd_data.head()


,data,version
0,{u'paragraphs': [{u'qas': [{u'question': u'To ...,1.1
1,{u'paragraphs': [{u'qas': [{u'question': u'Whe...,1.1
2,"{u'paragraphs': [{u'qas': [{u'question': u""Whe...",1.1
3,{u'paragraphs': [{u'qas': [{u'question': u'In ...,1.1
4,{u'paragraphs': [{u'qas': [{u'question': u'Wha...,1.1


In [4]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/home/anjana/Downloads/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
stopword = stopwords.words('english')
qtntype_list = ['How', 'What', 'When', 'Why', 'Which', 'Who','Whom', 'Where']
pun_list = list(string.punctuation)
delelements_list = stopword + qtntype_list + pun_list

In [6]:
qtntype_list


['How', 'What', 'When', 'Why', 'Which', 'Who', 'Whom', 'Where']

In [7]:
row_list = ['How many','How much','What','When','Where','Who', 'Which','Whom','PERSON',
          'NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
          'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE', 'ORG']


column_list = ['PERSON','NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
               'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE','ORG']


In [8]:
co_occ_matrix = np.zeros((len(row_list), len(column_list)), dtype = int)
co_occ_matrix

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
len(sqd_data)


442

In [20]:
for i in range(len(sqd_data)):
    print(i)
    paragraph_pointer = sqd_data.data[i]['paragraphs'] 

    #print(i, " ", len(paragraph_pointer))
    for j in range(len(paragraph_pointer)):#no of paragraphs in a row of the df
        processed_sen_list, process_sen_tokenized_list = preprocess_text(paragraph_pointer[j]['context']) 
        processed_que_list = preprocess_question(paragraph_pointer[j]['qas'])
        question_mapping = question_type(processed_que_list)
        que_type_extract = question_type_extractor(question_mapping)
        #print(que_type_extract)
        que_sen, ans_sen = wmd_que_ans_pair(processed_que_list, processed_sen_list)
        #print(wmd_dat)
#         print(que_sen)
#         print()
#         print(ans_sen)
        que_sen_map, ans_sen_map = que_ner_ans_ner(que_sen, ans_sen)
        que_no_quetag_df, que_no_anstag_df = que_df_ans_df(que_sen_map, ans_sen_map)
        
        for i in range(len(que_type_extract)):

            question_words = que_no_quetag_df[que_no_quetag_df['QuestionNo']== i]['QuestionTag'].tolist() + [que_type_extract[i]]
            ans_words = que_no_anstag_df[que_no_anstag_df['QuestionNo']==i]['AnswerTag'].tolist()
            combo = que_ans_combo_(question_words, ans_words)
            
            for i, v1 in enumerate(row_list):
                for j, v2 in enumerate(column_list):
                    for k in combo:
                        if [v1, v2] == k:
                            co_occ_matrix[i, j]+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

##### Definitions ##### 

In [11]:
def replace_all(repls, str):
   # return re.sub('|'.join(repls.keys()), lambda k: repls[k.group(0)], str)                                    
   return re.sub('|'.join(re.escape(key) for key in repls.keys()),
                 lambda k: repls[k.group(0)], str)

In [12]:
def preprocess_question(para_question):
    question_list = []
    for i in range(len(para_question)):
        l1 = para_question[i]['question']
        l1 = unicodedata.normalize('NFKD', l1).encode('ascii','ignore').decode('utf-8')
        question_list.append(l1)
    
    #Appropriate question type replacement
    Question_list = []
    for i in range(len(question_list)):
        Question_list.append(replace_all({"how": "How", "when": "When","what": "What","whom": "Whom","which": "Which"}, question_list[i]))
        
    return Question_list

In [13]:
def preprocess_text(para_context):

    context = unicodedata.normalize('NFKD', para_context).encode('ascii','ignore').decode('utf-8')
    #print(context)
    #sentence_tokeniztion
    senttokenized_context = sent_tokenize(context)
    sentencelist = []
    for sent_token in senttokenized_context:
        sentencelist.append(sent_token) 
    
    sentence_list = []
    lemmatizer = WordNetLemmatizer()


    for i in sentencelist:
        words = nltk.word_tokenize(i)
        words = ([x.lower() for x in words])
        words = ([x for x in words if x not in delelements_list])
        words = [lemmatizer.lemmatize(x) for x in words]
        words = [[s.encode('ascii').decode('utf-8') for s in words]]
        sentence_list.append(words)
    return sentencelist, sentence_list

In [14]:
def question_type_extractor(question_mapping):
    Question_mapping = []
    for i in range(len(question_mapping)):
        if question_mapping[i] != 'How many' and question_mapping[i] != 'How much':
            Question_mapping.append(question_mapping[i].split()[0])
        else:
             Question_mapping.append(question_mapping[i])
    return Question_mapping

In [15]:
def wmd_que_ans_pair(que_list, sen_list):
    wmd_matrix = np.zeros([len(que_list),len(sen_list)]) 
    for i in range(len(que_list)):
        for j in range(len(sen_list)):
            wmd_matrix[i,j] =  word2vec_model.wmdistance(que_list[i],sen_list[j])  
    wmd_df = pd.DataFrame(wmd_matrix)
    wmd_df['sentence'] = wmd_df.T.idxmin()
    wmd_df['qnno'] = wmd_df.index
    que_no_ans_sen_no = []
    for i in range(len(wmd_df)):
        que_no_ans_sen_no.append([wmd_df['qnno'][i], wmd_df['sentence'][i]])
    qn_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        if que_no_ans_sen_no[i][0] == que_list.index(que_list[i]):
            qn_sentence.append(que_list[i])
    an_sentence = []
    for i in range(len(que_no_ans_sen_no)):
        for j in range(len(sen_list)):
            if que_no_ans_sen_no[i][1] == sen_list.index(sen_list[j]):
                an_sentence.append(sen_list[j]) 
    return qn_sentence, an_sentence


In [16]:
def que_ner_ans_ner(que_sentence, ans_sentence):
    ans_sentence_mapping = []
    for i in range(len(ans_sentence)):
        doc_ans = nlp(ans_sentence[i])
        for ent in doc_ans.ents:
            ans_sentence_mapping.append([i,ent.label_])

    que_sentence_mapping = []
    for i in range(len(que_sentence)):
        doc_que = nlp(que_sentence[i])
        for ent in doc_que.ents:
            que_sentence_mapping.append([i,ent.label_])
    return que_sentence_mapping, ans_sentence_mapping


In [17]:
def que_df_ans_df(que_sen_map, ans_sen_map):
    qno_anstag_df = pd.DataFrame(ans_sen_map,columns = ['QuestionNo','AnswerTag'])
    qno_quetag_df = pd.DataFrame(que_sen_map, columns = ['QuestionNo', 'QuestionTag'])
    return qno_quetag_df, qno_anstag_df

In [18]:
def que_ans_combo_(questionword, answerword):
    ques_ans_combo = []
    for i in range(len(questionword)):
        for j in range(len(answerword)):
            ques_ans_combo.append([questionword[i],answerword[j]])
    return(ques_ans_combo)

In [19]:
def question_type(process_que_list):
    question_mapping = [] 
    for i in range(len(process_que_list)):
        l2 = nltk.word_tokenize(process_que_list[i])
        #print(len(l2))
        #print(l2)
        missed = l2[-1]        
        if missed == '?':
            for j in range(len(l2)):


                if l2[j] in qtntype_list:
                    combine_list = l2[j]+" "+l2[j+1]
                    question_mapping.append(combine_list)

        else:

             for k in range(len(l2)):#8

                if k < len(l2)-1:  #7        

                    if l2[k] in qtntype_list:
                        combine_list = l2[k]+" "+l2[k+1]
                        question_mapping.append(combine_list) 
                else:

                        if l2[k] in qtntype_list:
                            combine_list = l2[k]+" "+'abc'
                            question_mapping.append(combine_list) 
    return question_mapping

##### Co occurance matrix #####

In [21]:
co_occ_matrix_df = pd.DataFrame(co_occ_matrix, index = row_list, columns= column_list)
co_occ_matrix_df

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG
How many,1595,1513,167,418,30,149,173,52,2732,114,337,233,487,5384,3530,3545
How much,322,271,20,83,14,26,18,19,727,24,337,350,80,531,654,693
What,20895,15976,1479,4525,463,1304,2089,827,19422,657,2128,859,3995,12833,29214,33155
When,3233,2241,184,589,62,253,301,98,4711,98,53,95,790,1687,4288,4545
Where,1748,1045,262,490,33,104,128,36,1234,28,35,32,347,825,3075,2656
Who,7601,3114,211,629,83,263,479,90,3784,81,79,92,929,1791,4901,6460
Which,3945,2307,335,836,85,211,358,110,2799,78,151,124,686,1745,4924,5025
Whom,295,135,9,25,1,12,15,5,157,1,4,3,40,61,219,267
PERSON,19429,4240,482,1130,154,392,1075,232,7317,214,200,235,1403,3736,8327,9640
NORP,4628,12358,212,1319,115,335,417,402,4263,93,707,131,844,2821,8294,5916


#####  Heat map #####

In [22]:
co_occ_matrix_df['Tags'] = co_occ_matrix_df.index


In [23]:
#heatmap
melt_co_occ_matrix_df = pd.melt(co_occ_matrix_df,id_vars = ['Tags'])
melt_co_occ_matrix_df.head()


import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Heatmap(z=melt_co_occ_matrix_df['value'],
                  x=melt_co_occ_matrix_df['Tags'] ,
                  y=melt_co_occ_matrix_df['variable'])
data=[trace]
py.iplot(data)

In [24]:
from sklearn.preprocessing import normalize


In [25]:
normed_matrix = normalize(co_occ_matrix, axis=1, norm='l1')    #### dividing by sum of row...
normed_matrix

/home/anjana/virtualenvs/testsum/local/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning:

Data with input dtype int64 was converted to float64 by the normalize function.



array([[ 0.0779608 ,  0.07395278,  0.00816267,  0.02043111,  0.00146635,
         0.00728286,  0.00845594,  0.00254167,  0.13353536,  0.00557212,
         0.01647197,  0.01138863,  0.0238037 ,  0.26316047,  0.1725402 ,
         0.17327338],
       [ 0.07723675,  0.0650036 ,  0.00479731,  0.01990885,  0.00335812,
         0.00623651,  0.00431758,  0.00455745,  0.17438235,  0.00575678,
         0.08083473,  0.08395299,  0.01918925,  0.12736867,  0.15687215,
         0.16622691],
       [ 0.13946643,  0.10663392,  0.00987178,  0.03020271,  0.00309035,
         0.00870372,  0.01394331,  0.00551992,  0.1296347 ,  0.00438523,
         0.01420362,  0.00573351,  0.02666515,  0.08565555,  0.19499269,
         0.22129741],
       [ 0.13918547,  0.09647839,  0.00792147,  0.02535733,  0.00266919,
         0.01089203,  0.0129585 ,  0.00421905,  0.20281557,  0.00421905,
         0.00228173,  0.00408989,  0.03401068,  0.07262786,  0.18460479,
         0.19566902],
       [ 0.14472595,  0.08652095,  0

In [26]:
normed_matrix_df = pd.DataFrame(normed_matrix, index = row_list, columns= column_list)

normed_matrix_df['Tags'] = normed_matrix_df.index


In [27]:
#normalized heatmap
melt_normed_matrix_df = pd.melt(normed_matrix_df,id_vars = ['Tags'])
melt_normed_matrix_df.head()


import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Heatmap(z=melt_normed_matrix_df['value'],
                  x=melt_normed_matrix_df['Tags'] ,
                  y=melt_normed_matrix_df['variable'])
data=[trace]
py.iplot(data)

In [28]:
melt_normed_matrix_df.columns = ['Question_tag', 'Answer_tag', 'z_value']
melt_normed_matrix_df.head()

,Question_tag,Answer_tag,z_value
0,How many,PERSON,0.077961
1,How much,PERSON,0.077237
2,What,PERSON,0.139466
3,When,PERSON,0.139185
4,Where,PERSON,0.144726


In [29]:
answer_dict = {'LOC': 1,'PERSON': 2,'NORP': 3,'FAC': 4,'PRODUCT': 5,'EVENT': 6,'WORK_OF_ART': 7,'LANGUAGE': 8,'DATE':9,'TIME': 10,'PERCENT': 11,'MONEY': 12,'ORDINAL': 13,'CARDINAL': 14,'GPE': 15, 'ORG': 16}


In [30]:
answer_dict

{'CARDINAL': 14,
 'DATE': 9,
 'EVENT': 6,
 'FAC': 4,
 'GPE': 15,
 'LANGUAGE': 8,
 'LOC': 1,
 'MONEY': 12,
 'NORP': 3,
 'ORDINAL': 13,
 'ORG': 16,
 'PERCENT': 11,
 'PERSON': 2,
 'PRODUCT': 5,
 'TIME': 10,
 'WORK_OF_ART': 7}

In [31]:
question_dict = {'LOC': 10,
                 'PERSON': 20,
                 'NORP': 30,
                 'FAC': 40,
                 'PRODUCT': 50,
                 'EVENT':60,
                 'WORK_OF_ART':70,
                 'LANGUAGE':80,
                 'DATE': 90,
                 'TIME': 100,
                 'PERCENT': 110,
                 'MONEY': 120,
                 'ORDINAL':130,
                 'CARDINAL': 140,
                 'GPE': 150,
                 'ORG': 160,
                 'How many': 170,
                 'How much': 180,
                 'What': 190,
                 'When': 200,
                 'Where': 210,
                 'Who': 220,
                 'Which':230,
                 'Whom': 240}

In [32]:
question_dict

{'CARDINAL': 140,
 'DATE': 90,
 'EVENT': 60,
 'FAC': 40,
 'GPE': 150,
 'How many': 170,
 'How much': 180,
 'LANGUAGE': 80,
 'LOC': 10,
 'MONEY': 120,
 'NORP': 30,
 'ORDINAL': 130,
 'ORG': 160,
 'PERCENT': 110,
 'PERSON': 20,
 'PRODUCT': 50,
 'TIME': 100,
 'WORK_OF_ART': 70,
 'What': 190,
 'When': 200,
 'Where': 210,
 'Which': 230,
 'Who': 220,
 'Whom': 240}

In [33]:
#created a copy of melt_normed_matrix_df
melt_normed_matrix_df_1 = melt_normed_matrix_df.copy()
melt_normed_matrix_df_1.head()

,Question_tag,Answer_tag,z_value
0,How many,PERSON,0.077961
1,How much,PERSON,0.077237
2,What,PERSON,0.139466
3,When,PERSON,0.139185
4,Where,PERSON,0.144726


In [34]:
melt_normed_matrix_df_1["Question_tag"].replace(question_dict, inplace=True)
melt_normed_matrix_df_1['Answer_tag'].replace(answer_dict, inplace = True)


In [35]:
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy, Reader 

In [36]:
reader = Reader(line_format='user item rating', sep='\t')
melt_normed_matrix_df_1.head()


,Question_tag,Answer_tag,z_value
0,170,2,0.077961
1,180,2,0.077237
2,190,2,0.139466
3,200,2,0.139185
4,210,2,0.144726


In [79]:
csv_file  = melt_normed_matrix_df_1.to_csv("melt.txt", sep= '\t', header = False, index = False)
melt = Dataset.load_from_file("melt.txt", reader)
len(melt.raw_ratings)


384

In [80]:
type(melt)

instance

In [81]:
reconstruction_error = []
for i in range(20):  
   
   algo = SVDpp(n_factors=i)
   
   for trainset, testset in melt.folds():
       algo.train(trainset)
       predictions_svdpp = algo.test(testset)
       
   
   reconstruct = np.dot(algo.pu, algo.qi.T)
   rmse = mean_squared_error(normed_matrix, reconstruct)**0.5
   reconstruction_error.append(rmse)
   print(i, rmse)

(0, 0.097098650275542153)
(1, 0.096761331593372607)
(2, 0.097802321051397231)
(3, 0.098037763568412845)
(4, 0.098814256246885057)
(5, 0.099979413020117272)
(6, 0.1020594787873533)
(7, 0.10048029324525647)
(8, 0.098981383387766991)
(9, 0.10045395213473493)
(10, 0.10185124343237081)
(11, 0.10319819369418982)
(12, 0.10172993768683462)
(13, 0.10225541829189641)
(14, 0.10366757726617751)
(15, 0.10210163274680445)
(16, 0.10279824102704092)
(17, 0.096957737187905624)
(18, 0.10371740801932687)
(19, 0.10232378634200792)


In [140]:
algo = SVDpp(n_factors=10)
melt.split(n_folds=2) 


In [141]:
for trainset, testset in melt.folds():
    algo.train(trainset)
    predictions_svdpp = algo.test(testset)
    rmse = accuracy.rmse(predictions_svdpp, verbose=True) 

RMSE: 0.9424
RMSE: 0.9385


In [40]:
algo.pu.shape #question_type latent factor matrix


(24, 10)

In [41]:

algo.qi.T.shape #answer_type latent factor matrix


(10, 16)

In [42]:

reconstruct = np.dot(algo.pu, algo.qi.T)
reconstruct

array([[  2.40045831e-02,   1.74971551e-02,   1.81786659e-02,
          3.23232348e-02,   5.02806395e-02,  -3.77693954e-02,
         -1.75394291e-02,   3.47533047e-03,  -1.58846271e-02,
         -2.01862137e-02,  -2.70051786e-02,  -6.58979712e-02,
          6.59311194e-02,   1.35342664e-02,   5.30302676e-03,
          2.05442049e-03],
       [ -1.21966090e-05,   1.68095181e-02,  -5.03502892e-03,
          6.85578875e-04,  -4.03357454e-02,  -4.43868041e-02,
         -7.56064996e-03,   2.62076805e-02,  -4.74385656e-02,
         -1.59584269e-02,  -2.96985887e-02,  -4.92118009e-02,
          2.20905806e-02,  -1.72613235e-02,  -1.27738656e-02,
         -3.00183674e-02],
       [ -4.97627195e-02,  -1.36100499e-02,   3.11792116e-02,
         -4.35178023e-02,   6.92652748e-03,  -1.20994181e-02,
         -3.71285909e-03,  -3.95418026e-02,  -4.32761580e-02,
          1.20275544e-02,  -2.05259505e-02,  -2.32373104e-02,
          6.88978476e-03,  -2.01788401e-02,   2.42381725e-02,
          7.6821

In [43]:
normed_matrix


array([[ 0.0779608 ,  0.07395278,  0.00816267,  0.02043111,  0.00146635,
         0.00728286,  0.00845594,  0.00254167,  0.13353536,  0.00557212,
         0.01647197,  0.01138863,  0.0238037 ,  0.26316047,  0.1725402 ,
         0.17327338],
       [ 0.07723675,  0.0650036 ,  0.00479731,  0.01990885,  0.00335812,
         0.00623651,  0.00431758,  0.00455745,  0.17438235,  0.00575678,
         0.08083473,  0.08395299,  0.01918925,  0.12736867,  0.15687215,
         0.16622691],
       [ 0.13946643,  0.10663392,  0.00987178,  0.03020271,  0.00309035,
         0.00870372,  0.01394331,  0.00551992,  0.1296347 ,  0.00438523,
         0.01420362,  0.00573351,  0.02666515,  0.08565555,  0.19499269,
         0.22129741],
       [ 0.13918547,  0.09647839,  0.00792147,  0.02535733,  0.00266919,
         0.01089203,  0.0129585 ,  0.00421905,  0.20281557,  0.00421905,
         0.00228173,  0.00408989,  0.03401068,  0.07262786,  0.18460479,
         0.19566902],
       [ 0.14472595,  0.08652095,  0

In [416]:
reconstruct_error = normed_matrix - reconstruct
reconstruct_error

array([[  8.88030375e-02,   1.54627739e-03,   7.51757094e-02,
          2.34215149e-02,  -5.68896277e-03,  -1.30951062e-02,
          1.20135926e-02,  -5.08364899e-02,   1.73237297e-01,
          5.73409363e-03,   2.73466793e-02,  -8.36085273e-02,
         -1.33817735e-02,   2.21086390e-01,   1.33574668e-01,
          2.02092623e-01],
       [  8.63085385e-02,   6.55683646e-02,   8.44890975e-02,
          5.07827631e-02,   4.94300395e-03,   1.80779386e-02,
          2.09230167e-02,   3.83449658e-02,   1.98310510e-01,
         -2.23060929e-02,   6.29189730e-02,   1.01108640e-01,
          2.35304479e-02,   1.13869753e-01,   1.05408136e-01,
          1.06266614e-01],
       [  1.76407051e-01,   1.08788060e-01,   1.97236746e-02,
          9.03064274e-03,  -1.86363156e-02,   3.24616741e-02,
         -2.09294931e-04,   3.85094079e-02,   1.25533539e-01,
          2.31123715e-02,   6.18064697e-02,   3.67185333e-02,
          9.30533861e-02,   8.87730246e-02,   1.85412530e-01,
          2.0816

In [417]:
import math
from sklearn.metrics import mean_squared_error


In [418]:
rmse = mean_squared_error(normed_matrix, reconstruct)**0.5
rmse

0.10702285139371678

In [419]:
reconstruct_error_df = pd.DataFrame(reconstruct_error,index = row_list,columns = column_list)
reconstruct_error_df

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG
How many,0.088803,0.001546,0.075176,0.023422,-0.005689,-0.013095,0.012014,-0.050836,0.173237,0.005734,0.027347,-0.083609,-0.013382,0.221086,0.133575,0.202093
How much,0.086309,0.065568,0.084489,0.050783,0.004943,0.018078,0.020923,0.038345,0.198311,-0.022306,0.062919,0.101109,0.023530,0.113870,0.105408,0.106267
What,0.176407,0.108788,0.019724,0.009031,-0.018636,0.032462,-0.000209,0.038509,0.125534,0.023112,0.061806,0.036719,0.093053,0.088773,0.185413,0.208167
When,0.283020,0.111368,0.006617,0.000127,0.003635,0.001168,0.008177,0.029442,0.203645,0.039498,0.059961,-0.077343,0.111837,0.109523,0.171350,0.209803
Where,0.094746,0.019904,0.059534,0.117269,-0.026387,-0.033657,0.029917,0.035550,0.267277,0.000520,-0.057086,0.036537,0.087712,0.072544,0.248634,0.160064
Who,0.297692,0.147451,0.035257,0.015767,0.030551,0.016838,0.026542,0.051130,0.117068,-0.019817,0.034823,0.021336,0.028985,0.100792,0.183680,0.248039
Which,0.124458,0.016164,-0.012624,-0.012612,0.058464,0.019389,-0.002226,0.015074,0.177462,0.058763,-0.018447,0.054027,0.086787,0.146150,0.242396,0.189510
Whom,0.216040,0.099900,-0.028463,0.054718,0.060185,0.006680,-0.019963,0.033671,0.153286,-0.001841,0.009648,0.086946,0.050720,0.116694,0.153933,0.230815
PERSON,0.305314,0.155937,0.015663,0.069432,-0.011319,0.001675,0.040858,-0.003428,0.121475,-0.037244,-0.038862,-0.023259,-0.019480,0.056913,0.135509,0.196760
NORP,0.086500,0.212898,0.017138,0.000383,-0.087215,-0.041235,-0.001714,0.037340,0.078712,0.062605,0.036742,-0.008344,-0.031265,0.044775,0.211188,0.111519


In [421]:
import plotly.graph_objs as go

trace1 = go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['PERSON'][0:8],
    name='PERSON'
)
trace2 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['NORP'][0:8],
    name='NORP'
)
trace3 = go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['FAC'][0:8],
    name='FAC'
)
trace4 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['LOC'][0:8],
    name='LOC'
)


trace5 = go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['PRODUCT'][0:8],
    name='PRODUCT'
)
trace6 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['EVENT'][0:8],
    name='EVENT'
)


trace7= go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['WORK_OF_ART'][0:8],
    name='WORK_OF_ART'
)
trace8 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['LANGUAGE'][0:8],
    name='LANGUAGE'
)



trace9 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['DATE'][0:8],
    name='DATE'
)
trace10= go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['TIME'][0:8],
    name='TIME'
)
trace11 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['PERCENT'][0:8],
    name='PERCENT'
)



trace12 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['MONEY'][0:8],
    name='MONEY'
)
trace13= go.Bar(
    x=row_list[0:8],
    y=reconstruct_error_df['ORDINAL'][0:8],
    name='ORDINAL'
)
trace14 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['CARDINAL'][0:8],
    name='CARDINAL'
)


trace15= go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['GPE'][0:8],
    name='GPE'
)
trace16 = go.Bar(
    x=row_list[0:8],
    y= reconstruct_error_df['ORG'][0:8],
    name='ORG'
)



data = [trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10,trace11,
        trace12,trace13,trace14,trace15,trace16]
layout = go.Layout(
    barmode='group'
) 



In [423]:
#fig = go.Figure(data=data, layout=layout)
#py.iplot(fig)

In [47]:
normed_matrix

array([[ 0.0779608 ,  0.07395278,  0.00816267,  0.02043111,  0.00146635,
         0.00728286,  0.00845594,  0.00254167,  0.13353536,  0.00557212,
         0.01647197,  0.01138863,  0.0238037 ,  0.26316047,  0.1725402 ,
         0.17327338],
       [ 0.07723675,  0.0650036 ,  0.00479731,  0.01990885,  0.00335812,
         0.00623651,  0.00431758,  0.00455745,  0.17438235,  0.00575678,
         0.08083473,  0.08395299,  0.01918925,  0.12736867,  0.15687215,
         0.16622691],
       [ 0.13946643,  0.10663392,  0.00987178,  0.03020271,  0.00309035,
         0.00870372,  0.01394331,  0.00551992,  0.1296347 ,  0.00438523,
         0.01420362,  0.00573351,  0.02666515,  0.08565555,  0.19499269,
         0.22129741],
       [ 0.13918547,  0.09647839,  0.00792147,  0.02535733,  0.00266919,
         0.01089203,  0.0129585 ,  0.00421905,  0.20281557,  0.00421905,
         0.00228173,  0.00408989,  0.03401068,  0.07262786,  0.18460479,
         0.19566902],
       [ 0.14472595,  0.08652095,  0

##### PART -III #####

##### (1-normed_matrix)#####

In [108]:
dissimilar_normed_matrix = 1 - normed_matrix
dissimilar_normed_matrix 

array([[ 0.9220392 ,  0.92604722,  0.99183733,  0.97956889,  0.99853365,
         0.99271714,  0.99154406,  0.99745833,  0.86646464,  0.99442788,
         0.98352803,  0.98861137,  0.9761963 ,  0.73683953,  0.8274598 ,
         0.82672662],
       [ 0.92276325,  0.9349964 ,  0.99520269,  0.98009115,  0.99664188,
         0.99376349,  0.99568242,  0.99544255,  0.82561765,  0.99424322,
         0.91916527,  0.91604701,  0.98081075,  0.87263133,  0.84312785,
         0.83377309],
       [ 0.86053357,  0.89336608,  0.99012822,  0.96979729,  0.99690965,
         0.99129628,  0.98605669,  0.99448008,  0.8703653 ,  0.99561477,
         0.98579638,  0.99426649,  0.97333485,  0.91434445,  0.80500731,
         0.77870259],
       [ 0.86081453,  0.90352161,  0.99207853,  0.97464267,  0.99733081,
         0.98910797,  0.9870415 ,  0.99578095,  0.79718443,  0.99578095,
         0.99771827,  0.99591011,  0.96598932,  0.92737214,  0.81539521,
         0.80433098],
       [ 0.85527405,  0.91347905,  0

##### SVD for dissimilar_normed_matrix #####

In [109]:


dissimilar_normed_matrix_df = pd.DataFrame(dissimilar_normed_matrix, index = row_list, columns= column_list)

dissimilar_normed_matrix_df['Tags'] = dissimilar_normed_matrix_df.index


In [110]:
dissimilar_normed_matrix_df

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG,Tags
How many,0.922039,0.926047,0.991837,0.979569,0.998534,0.992717,0.991544,0.997458,0.866465,0.994428,0.983528,0.988611,0.976196,0.736840,0.827460,0.826727,How many
How much,0.922763,0.934996,0.995203,0.980091,0.996642,0.993763,0.995682,0.995443,0.825618,0.994243,0.919165,0.916047,0.980811,0.872631,0.843128,0.833773,How much
What,0.860534,0.893366,0.990128,0.969797,0.996910,0.991296,0.986057,0.994480,0.870365,0.995615,0.985796,0.994266,0.973335,0.914344,0.805007,0.778703,What
When,0.860815,0.903522,0.992079,0.974643,0.997331,0.989108,0.987042,0.995781,0.797184,0.995781,0.997718,0.995910,0.965989,0.927372,0.815395,0.804331,When
Where,0.855274,0.913479,0.978308,0.959430,0.997268,0.991389,0.989402,0.997019,0.897831,0.997682,0.997102,0.997351,0.971270,0.931694,0.745405,0.780096,Where
Who,0.751496,0.898192,0.993102,0.979436,0.997286,0.991402,0.984340,0.997058,0.876287,0.997352,0.997417,0.996992,0.969628,0.941446,0.839769,0.788799,Who
Which,0.833678,0.902736,0.985876,0.964754,0.996416,0.991104,0.984907,0.995362,0.881993,0.996711,0.993634,0.994772,0.971078,0.926430,0.792403,0.788145,Which
Whom,0.763811,0.891914,0.992794,0.979984,0.999199,0.990392,0.987990,0.995997,0.874299,0.999199,0.996797,0.997598,0.967974,0.951161,0.824660,0.786229,Whom
PERSON,0.666203,0.927155,0.991719,0.980586,0.997354,0.993265,0.981531,0.996014,0.874291,0.996323,0.996564,0.995963,0.975896,0.935814,0.856939,0.834381,PERSON
NORP,0.892008,0.711632,0.995053,0.969222,0.997317,0.992183,0.990270,0.990620,0.900525,0.997830,0.983503,0.996943,0.980306,0.934173,0.806464,0.861953,NORP


##### Melting dissimilar_normed_matrix_df ######


In [111]:

melt_dissimilar_normed_matrix_df = pd.melt(dissimilar_normed_matrix_df,id_vars = ['Tags'])
melt_dissimilar_normed_matrix_df.columns = ['Question_tag', 'Answer_tag', 'z_value']
melt_dissimilar_normed_matrix_df.head()

,Question_tag,Answer_tag,z_value
0,How many,PERSON,0.922039
1,How much,PERSON,0.922763
2,What,PERSON,0.860534
3,When,PERSON,0.860815
4,Where,PERSON,0.855274


In [112]:
reader_new = Reader(line_format='user item rating', sep='\t', rating_scale=(0,1))

csv_file  = melt_dissimilar_normed_matrix_df.to_csv("melt_dissimilar.txt", sep= '\t', header = False, index = False)
melt_dissimilar = Dataset.load_from_file("melt_dissimilar.txt", reader_new)


##### Deciding the number of factors for SVDpp ##### 

In [407]:
dissimilar_rmse_df = []
melt_dissimilar.split(n_folds=2) 

for i in range(20):
    algo = SVDpp(n_factors=i) 
    for trainset, testset in melt_dissimilar.folds():
        algo.train(trainset) 
        predictions_svdpp = algo.test(testset)

    reconstruct = np.dot(algo.pu, algo.qi.T)
    reconstruct_error = dissimilar_normed_matrix - reconstruct
    rmse = mean_squared_error(dissimilar_normed_matrix, reconstruct)**0.5
    dissimilar_rmse_df.append([i,rmse]) 

In [408]:
dissimilar_rmse_df 

[[0, 0.94044040102780146],
 [1, 0.94260281052890482],
 [2, 0.94040954839749824],
 [3, 0.93944367131805184],
 [4, 0.94137829528551931],
 [5, 0.9399303756015146],
 [6, 0.94048844596624481],
 [7, 0.94109133182084836],
 [8, 0.9412533158877221],
 [9, 0.93913468584969284],
 [10, 0.93955566667697687],
 [11, 0.9403672285673591],
 [12, 0.9400671395462139],
 [13, 0.940883506720948],
 [14, 0.93931225894637271],
 [15, 0.94178803952175116],
 [16, 0.94365880313045447],
 [17, 0.93870612434586287],
 [18, 0.94022325922524896],
 [19, 0.94392389902943896]]

Number of factors = 9

In [142]:
algo_dissimilar = SVDpp(n_factors=9)
melt_dissimilar.split(n_folds=2) 



In [145]:

for trainset, testset in melt_dissimilar.folds():
    algo_dissimilar.train(trainset)
    predictions_svdpp = algo_dissimilar.test(testset)
    rmse = accuracy.rmse(predictions_svdpp, verbose=True) 

RMSE: 0.0475
RMSE: 0.0612


In [120]:
print(algo_dissimilar.pu.shape )#question_type latent factor matrix

print(algo_dissimilar.qi.T.shape) #answer_type latent factor matrix

dissimilar_reconstruct = np.dot(algo_dissimilar.pu, algo_dissimilar.qi.T)
dissimilar_reconstruct

(24, 9)
(9, 16)


array([[-0.02441779, -0.00501271, -0.01368152, -0.01130229,  0.00288956,
         0.00960247, -0.02658632,  0.00291906, -0.0138603 ,  0.00825451,
        -0.00268164, -0.01393396,  0.00548327, -0.02605475, -0.00827585,
        -0.00924625],
       [-0.05440273,  0.04454658,  0.00697832, -0.01278731,  0.01688655,
        -0.00234862,  0.0607842 ,  0.01653028,  0.00675886, -0.03576205,
         0.04310674,  0.00595407,  0.04074044, -0.00850557,  0.00869627,
        -0.01533801],
       [ 0.02373288,  0.01602754, -0.00458256, -0.05020884, -0.00326659,
        -0.03719945,  0.04048438,  0.00048823,  0.03560778, -0.0501471 ,
         0.02380696,  0.04918199,  0.02489236,  0.03319952,  0.03372208,
         0.04106115],
       [-0.00661508,  0.05630986, -0.07293215, -0.03537068, -0.00221976,
         0.08245774, -0.01737605, -0.0055457 ,  0.0583307 , -0.05240376,
        -0.01791317, -0.0966854 ,  0.02725313, -0.00280341, -0.00072828,
        -0.04922899],
       [-0.0389535 , -0.00873334, -0

In [409]:
dissimilar_reconstruct_error = dissimilar_normed_matrix - dissimilar_reconstruct


In [410]:
dissimilar_reconstruct_error_df = pd.DataFrame(dissimilar_reconstruct_error, index = row_list, columns= column_list)

dissimilar_reconstruct_error_df['Tags'] = dissimilar_normed_matrix_df.index
 

In [411]:
dissimilar_reconstruct_error_df 

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG,Tags
How many,0.946457,0.931060,1.005519,0.990871,0.995644,0.983115,1.018130,0.994539,0.880325,0.986173,0.986210,1.002545,0.970713,0.762894,0.835736,0.835973,How many
How much,0.977166,0.890450,0.988224,0.992878,0.979755,0.996112,0.934898,0.978912,0.818859,1.030005,0.876059,0.910093,0.940070,0.881137,0.834432,0.849111,How much
What,0.836801,0.877339,0.994711,1.020006,1.000176,1.028496,0.945572,0.993992,0.834758,1.045762,0.961989,0.945084,0.948442,0.881145,0.771285,0.737641,What
When,0.867430,0.847212,1.065011,1.010013,0.999551,0.906650,1.004418,1.001327,0.738854,1.048185,1.015631,1.092596,0.938736,0.930176,0.816123,0.853560,When
Where,0.894228,0.922212,0.993752,0.970626,0.983732,1.008271,1.027857,0.975826,0.944268,1.036008,0.977211,0.974812,0.955249,0.982111,0.717188,0.770156,Where
Who,0.714961,0.944788,1.016462,0.899545,1.006842,1.054423,1.023223,0.928117,0.914058,0.982834,0.995128,0.948800,0.990811,0.962554,0.819564,0.769928,Who
Which,0.838752,0.886430,0.998999,1.011208,0.969009,0.955896,0.974948,1.047407,0.860444,0.974106,1.024667,1.089660,0.974145,0.848360,0.856619,0.845680,Which
Whom,0.787714,0.882509,0.969291,0.961064,0.998210,0.990756,1.025597,1.017364,0.888888,1.011359,0.978410,1.015396,0.971469,0.961042,0.766088,0.796612,Whom
PERSON,0.652900,0.919854,1.003376,1.047790,1.006256,0.947938,0.966225,1.027313,0.825312,0.963072,1.032271,1.029234,0.972762,0.919595,0.911705,0.837199,PERSON
NORP,0.892387,0.713605,1.002009,0.928541,1.005843,1.061885,0.923286,0.973394,0.888726,0.961360,0.941541,0.966549,0.965860,0.865473,0.837895,0.869437,NORP


##### Error Plots ######

In [412]:
import plotly.graph_objs as go

trace1 = go.Bar(
    x=row_list[0:8],
    y=dissimilar_reconstruct_error_df['PERSON'][0:8],
    name='PERSON'
)
trace2 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['NORP'][0:8],
    name='NORP'
)
trace3 = go.Bar(
    x=row_list[0:8],
    y=dissimilar_reconstruct_error_df['FAC'][0:8],
    name='FAC'
)
trace4 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['LOC'][0:8],
    name='LOC'
)


trace5 = go.Bar(
    x=row_list[0:8],
    y=dissimilar_reconstruct_error_df['PRODUCT'][0:8],
    name='PRODUCT'
)
trace6 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['EVENT'][0:8],
    name='EVENT'
)


trace7= go.Bar(
    x=row_list[0:8],
    y=dissimilar_reconstruct_error_df['WORK_OF_ART'][0:8],
    name='WORK_OF_ART'
)
trace8 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['LANGUAGE'][0:8],
    name='LANGUAGE'
)



trace9 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['DATE'][0:8],
    name='DATE'
)
trace10= go.Bar(
    x=row_list[0:8],
    y=dissimilar_reconstruct_error_df['TIME'][0:8],
    name='TIME'
)
trace11 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['PERCENT'][0:8],
    name='PERCENT'
)



trace12 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['MONEY'][0:8],
    name='MONEY'
)
trace13= go.Bar(
    x=row_list[0:8],
    y=dissimilar_reconstruct_error_df['ORDINAL'][0:8],
    name='ORDINAL'
)
trace14 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['CARDINAL'][0:8],
    name='CARDINAL'
)


trace15= go.Bar(
    x=row_list[0:8],
    y=dissimilar_reconstruct_error_df['GPE'][0:8],
    name='GPE'
)
trace16 = go.Bar(
    x=row_list[0:8],
    y= dissimilar_reconstruct_error_df['ORG'][0:8],
    name='ORG'
)



data = [trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10,trace11,
        trace12,trace13,trace14,trace15,trace16]
layout = go.Layout(
    barmode='group'
) 



In [413]:
dissimilar_reconstruct_error_df.head()

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG,Tags
How many,0.946457,0.931060,1.005519,0.990871,0.995644,0.983115,1.018130,0.994539,0.880325,0.986173,0.986210,1.002545,0.970713,0.762894,0.835736,0.835973,How many
How much,0.977166,0.890450,0.988224,0.992878,0.979755,0.996112,0.934898,0.978912,0.818859,1.030005,0.876059,0.910093,0.940070,0.881137,0.834432,0.849111,How much
What,0.836801,0.877339,0.994711,1.020006,1.000176,1.028496,0.945572,0.993992,0.834758,1.045762,0.961989,0.945084,0.948442,0.881145,0.771285,0.737641,What
When,0.867430,0.847212,1.065011,1.010013,0.999551,0.906650,1.004418,1.001327,0.738854,1.048185,1.015631,1.092596,0.938736,0.930176,0.816123,0.853560,When
Where,0.894228,0.922212,0.993752,0.970626,0.983732,1.008271,1.027857,0.975826,0.944268,1.036008,0.977211,0.974812,0.955249,0.982111,0.717188,0.770156,Where


In [403]:
trace1 = go.Bar(
    x=row_list[8:16],
    y=dissimilar_reconstruct_error_df['PERSON'][8:16],
    name='PERSON'
)
trace2 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['NORP'][8:16],
    name='NORP'
)
trace3 = go.Bar(
    x=row_list[8:16],
    y=dissimilar_reconstruct_error_df['FAC'][8:16],
    name='FAC'
)
trace4 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['LOC'][8:16],
    name='LOC'
)


trace5 = go.Bar(
    x=row_list[8:16],
    y=dissimilar_reconstruct_error_df['PRODUCT'][8:16],
    name='PRODUCT'
)
trace6 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['EVENT'][8:16],
    name='EVENT'
)


trace7= go.Bar(
    x=row_list[8:16],
    y=dissimilar_reconstruct_error_df['WORK_OF_ART'][8:16],
    name='WORK_OF_ART'
)
trace8 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['LANGUAGE'][8:16],
    name='LANGUAGE'
)



trace9 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['DATE'][8:16],
    name='DATE'
)
trace10= go.Bar(
    x=row_list[8:16],
    y=dissimilar_reconstruct_error_df['TIME'][8:16],
    name='TIME'
)
trace11 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['PERCENT'][8:16],
    name='PERCENT'
)



trace12 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['MONEY'][8:16],
    name='MONEY'
)
trace13= go.Bar(
    x=row_list[8:16],
    y=dissimilar_reconstruct_error_df['ORDINAL'][8:16],
    name='ORDINAL'
)
trace14 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['CARDINAL'][8:16],
    name='CARDINAL'
)


trace15= go.Bar(
    x=row_list[8:16],
    y=dissimilar_reconstruct_error_df['GPE'][8:16],
    name='GPE'
)
trace16 = go.Bar(
    x=row_list[8:16],
    y= dissimilar_reconstruct_error_df['ORG'][8:16],
    name='ORG'
)



data1 = [trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10,trace11,
        trace12,trace13,trace14,trace15,trace16]
layout = go.Layout(
    barmode='group'
) 



In [398]:
trace1 = go.Bar(
    x=row_list[16:24],
    y=dissimilar_reconstruct_error_df['PERSON'][16:24],
    name='PERSON'
)
trace2 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['NORP'][16:24],
    name='NORP'
)
trace3 = go.Bar(
    x=row_list[16:24],
    y=dissimilar_reconstruct_error_df['FAC'][16:24],
    name='FAC'
)
trace4 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['LOC'][16:24],
    name='LOC'
)


trace5 = go.Bar(
    x=row_list[16:24],
    y=dissimilar_reconstruct_error_df['PRODUCT'][16:24],
    name='PRODUCT'
)
trace6 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['EVENT'][16:24],
    name='EVENT'
)


trace7= go.Bar(
    x=row_list[16:24],
    y=dissimilar_reconstruct_error_df['WORK_OF_ART'][16:24],
    name='WORK_OF_ART'
)
trace8 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['LANGUAGE'][16:24],
    name='LANGUAGE'
)



trace9 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['DATE'][16:24],
    name='DATE'
)
trace10= go.Bar(
    x=row_list[16:24],
    y=dissimilar_reconstruct_error_df['TIME'][16:24],
    name='TIME'
)
trace11 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['PERCENT'][16:24],
    name='PERCENT'
)



trace12 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['MONEY'][16:24],
    name='MONEY'
)
trace13= go.Bar(
    x=row_list[16:24],
    y=dissimilar_reconstruct_error_df['ORDINAL'][16:24],
    name='ORDINAL'
)
trace14 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['CARDINAL'][16:24],
    name='CARDINAL'
)


trace15= go.Bar(
    x=row_list[16:24],
    y=dissimilar_reconstruct_error_df['GPE'][16:24],
    name='GPE'
)
trace16 = go.Bar(
    x=row_list[16:24],
    y= dissimilar_reconstruct_error_df['ORG'][16:24],
    name='ORG'
)



data2 = [trace1, trace2,trace3,trace4,trace5, trace6,trace7,trace8,trace9, trace10,trace11,
        trace12,trace13,trace14,trace15,trace16]
layout = go.Layout(
    barmode='group'
) 



##### Error distribution #####

In [404]:
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [405]:
fig = go.Figure(data=data1, layout=layout)
py.iplot(fig)

In [406]:
fig = go.Figure(data=data2, layout=layout)
py.iplot(fig)

##### Heat map for Error matrix #####

In [159]:
df = pd.melt(dissimilar_reconstruct_error_df,id_vars = ['Tags'])
df.columns = ['Question_tag', 'Answer_tag', 'z_value']
df.head()

,Question_tag,Answer_tag,z_value
0,How many,PERSON,0.946457
1,How much,PERSON,0.977166
2,What,PERSON,0.836801
3,When,PERSON,0.867430
4,Where,PERSON,0.894228


In [160]:
import plotly.plotly as py

from plotly.graph_objs import *

In [161]:
trace1 = Heatmap(
    z=df['z_value'] ,  # link 2D array
    x=df['Question_tag'],   # link x-axis labels
    y=df['Answer_tag'],            # link y-axis labels
    colorscale='Jet',   # (!) select pre-defined colormap
    reversescale=True      # (!) reverve color ordering
)

# Link it up in a data object
data = Data([trace1]) 

In [162]:
py.iplot(data)


##### Eucledian distance for all question tag / type- answer tag #####

In [387]:
from scipy.spatial import distance


In [388]:
qn_ans_list = list(algo.pu) + list(algo.qi)
qn_ans_list

qn_ans_listcopy = copy.copy(qn_ans_list)




In [389]:
question_answer_eucldist = np.zeros((40,40)) 

for i in range(len(qn_ans_list)):
    for j in range(len(qn_ans_listcopy)):
       # a = tuple(d[i]) 
        #b = tuple(D[i]) 
        #question_answer_eucldist[i,j] = distance.euclidean(a,b)
        
        question_answer_eucldist[i,j] = distance.euclidean(qn_ans_list[i],qn_ans_listcopy[j])
        #print(a,b)



In [390]:
row_column_list = row_list + column_list

In [391]:
algopu_algoqi_df = pd.DataFrame(question_answer_eucldist,index = row_column_list,columns = row_column_list)

In [392]:
algopu_algoqi_df

,How many,How much,What,When,Where,Who,Which,Whom,PERSON,NORP,...,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE,ORG
How many,0.000000,0.486803,0.536811,0.509886,0.495422,0.477907,0.276059,0.445776,0.335674,0.301122,...,0.542033,0.527508,0.388420,0.534491,0.354818,0.480593,0.361895,0.576422,0.443695,0.485685
How much,0.486803,0.000000,0.510656,0.687599,0.612475,0.512920,0.412961,0.551572,0.403788,0.414872,...,0.520302,0.560594,0.559349,0.657508,0.464665,0.393559,0.485425,0.419124,0.391744,0.423914
What,0.536811,0.510656,0.000000,0.560920,0.496031,0.442252,0.519600,0.516097,0.441572,0.435879,...,0.473561,0.277750,0.365820,0.412064,0.340837,0.338336,0.440568,0.468206,0.404320,0.476094
When,0.509886,0.687599,0.560920,0.000000,0.521091,0.536530,0.503284,0.469602,0.527302,0.472866,...,0.702500,0.400524,0.502095,0.440818,0.511228,0.582800,0.476686,0.663265,0.589475,0.551538
Where,0.495422,0.612475,0.496031,0.521091,0.000000,0.447683,0.473755,0.397003,0.444835,0.365154,...,0.647886,0.514645,0.483494,0.395605,0.410778,0.566546,0.477161,0.527833,0.405568,0.285913
Who,0.477907,0.512920,0.442252,0.536530,0.447683,0.000000,0.475849,0.363432,0.370176,0.304360,...,0.607840,0.508366,0.348820,0.431269,0.222060,0.371859,0.447808,0.518447,0.331506,0.396849
Which,0.276059,0.412961,0.519600,0.503284,0.473755,0.475849,0.000000,0.393814,0.370058,0.257365,...,0.547930,0.529677,0.436635,0.530563,0.356882,0.358106,0.450286,0.469924,0.361900,0.414001
Whom,0.445776,0.551572,0.516097,0.469602,0.397003,0.363432,0.393814,0.000000,0.417571,0.288217,...,0.464041,0.461954,0.426864,0.471178,0.306401,0.405829,0.357866,0.376854,0.309299,0.296787
PERSON,0.335674,0.403788,0.441572,0.527302,0.444835,0.370176,0.370058,0.417571,0.000000,0.290270,...,0.546944,0.507839,0.307565,0.401523,0.335067,0.407331,0.419766,0.484578,0.398269,0.336335
NORP,0.301122,0.414872,0.435879,0.472866,0.365154,0.304360,0.257365,0.288217,0.290270,0.000000,...,0.520363,0.453736,0.297341,0.477040,0.187320,0.346189,0.319088,0.445643,0.281401,0.292608
